In [111]:
#загрузка и установка библиотек
import pandas as pd
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
nltk.download('stopwords')
!pip install pymorphy2
import re
import pymorphy2
from collections import Counter 
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#загрузка данных
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283452 entries, 0 to 283451
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          283452 non-null  int64  
 1   title                       283452 non-null  object 
 2   short_description           133130 non-null  object 
 3   name_value_characteristics  50360 non-null   object 
 4   rating                      283452 non-null  float64
 5   feedback_quantity           283452 non-null  int64  
 6   category_id                 283452 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 15.1+ MB


In [ ]:
train.head()


,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790
3,1264824,Серьги,None,None,0.000000,0,14076
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401


In [ ]:
print('Количество уникальных заголовков:', train['title'].nunique())
print("Количество уникальных категорий:",train['category_id'].nunique())
print("Количество уникальных идентификаторов товара:",train['id'].nunique())

Количество уникальных заголовков: 239986
Количество уникальных категорий: 1231
Количество уникальных идентификаторов товара: 283452


In [112]:
#удаление дубликатов
train_drop = train.drop_duplicates(subset=['title', 'category_id'], keep="first")
train_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242720 entries, 0 to 283451
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          242720 non-null  int64  
 1   title                       242720 non-null  object 
 2   short_description           117302 non-null  object 
 3   name_value_characteristics  41276 non-null   object 
 4   rating                      242720 non-null  float64
 5   feedback_quantity           242720 non-null  int64  
 6   category_id                 242720 non-null  int64  
 7   new_title2                  242720 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 16.7+ MB


In [113]:
print('Количество уникальных заголовков:', fiks['title'].nunique())
print("Количество уникальных категорий:",fiks['category_id'].nunique())
print("Количество уникальных идентификаторов товара:",fiks['id'].nunique())

Количество уникальных заголовков: 239986
Количество уникальных категорий: 1231
Количество уникальных идентификаторов товара: 242720


In [85]:
#распределение по количеству товаров в каждой категории
distribute = pd.DataFrame(train[['category_id','title']].groupby('category_id')['title'].count()).sort_values('title', ascending=False)
distribute = distribute.reset_index()
distribute


,category_id,title
0,11937,14967
1,13408,7153
2,13061,6434
3,13143,6145
4,13253,3390
...,...,...
1226,12396,2
1227,2835,2
1228,2832,2
1229,14275,2


In [22]:
distribute['title'].describe().round()

count     1231.0
mean       230.0
std        657.0
min          2.0
25%         17.0
50%         60.0
75%        200.0
max      14967.0
Name: title, dtype: float64

Как видно классы несбалансированы, что может являться причиной неверной классификации редких классов. Уменьшим количество классов, число которых более 300, до 300 значений. И ответим на вопрос, повлияла ли такая эвристическая балансировка классов на точность модели.

In [114]:
#Определений категорий товаров в тестовом датасете кол-во которых более 300
name_category_for_train_balance = distribute[distribute['title'] >= 300]
name_category_for_train_balance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 0 to 217
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   category_id  218 non-null    int64
 1   title        218 non-null    int64
dtypes: int64(2)
memory usage: 5.1 KB


In [115]:
#cоздание "сбалансированного" датафрейма
train_balance = train
for i in name_category_for_train_balance['category_id']:
  count = int(name_category_for_train_balance[name_category_for_train_balance['category_id'] == i]['title'] - 300)
  idx = train_balance.index.values[train_balance['category_id'] == i][:count] 
  train_balance = train_balance.drop(idx)
train_balance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135838 entries, 3 to 283451
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          135838 non-null  int64  
 1   title                       135838 non-null  object 
 2   short_description           64795 non-null   object 
 3   name_value_characteristics  23716 non-null   object 
 4   rating                      135838 non-null  float64
 5   feedback_quantity           135838 non-null  int64  
 6   category_id                 135838 non-null  int64  
 7   new_title2                  135838 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 13.4+ MB


In [116]:
#Функции для обработки текста
#функция токенезации:
import re
def tokeniz(text):
  text = text.split()
  return text

#функция избавляющая от "лишних" символов
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
def regular(text):
  text = re.sub(patterns, '', text)
  return text

# функция лемматизации и удаления стоп-слов
import pymorphy2
stopwords_ru = stopwords.words("russian")
morph = pymorphy2.MorphAnalyzer()
def lematiz(text):
  new_text = []
  for word in text:
    if word not in stopwords_ru:
      word = morph.normal_forms(word)[0]
      new_text.append(word)
  return new_text

#функция для преобразования списка в строку
def string(text):
  text = ' '.join(text)
  return(text)

Дополнительный анализ слов, для улучшения модели.

Нахождение слов, которые не несут смысловую нагрузку, но встречаются часто.

In [117]:
#создание столбца с обработанными заголовками в виде списков для дальнейшей обработки
train['vocabluary'] = train['title'].apply(regular).apply(tokeniz).apply(lematiz)

In [118]:
train.head()

,id,title,short_description,name_value_characteristics,rating,feedback_quantity,category_id,new_title2,vocabluary
0,1267423,Muhle Manikure Песочные колпачки для педикюра ...,Muhle Manikure Колпачок песочный шлифовальный ...,None,0.000000,0,2693,песочный колпачок педикюр средний грита упаковка,"[песочный, колпачок, педикюр, средний, грита, ..."
1,128833,"Sony Xperia L1 Защитное стекло 2,5D",None,None,4.666667,9,13408,защитный стекло,"[защитный, стекло]"
2,569924,"Конверт для денег Прекрасная роза, 16,5 х 8 см","Конверт для денег «Прекрасная роза», 16,5 × 8 см",None,5.000000,6,11790,конверт деньга прекрасный роза смотреть,"[конверт, деньга, прекрасный, роза, х, смотреть]"
3,1264824,Серьги,None,None,0.000000,0,14076,серьга,[серьга]
4,1339052,Наклейки на унитаз для туалета на крышку бачок...,"Водостойкая, интересная наклейка на унитаз раз...",None,0.000000,0,12401,наклейка унитаз туалет крышка бачок дверь ванн...,"[наклейка, унитаз, туалет, крышка, бачок, двер..."


In [119]:
#объединение всех отфильтрованных слов в один список
vocabluary = []
for i in train['vocabluary']:
  vocabluary += i 


In [120]:
#подсчет количества вхождений каждого слова и сортировка по убыванию
vocabluary_count = Counter(vocabluary)
vocabluary_count = pd.DataFrame.from_dict(vocabluary_count, orient='index').reset_index()
vocabluary_count.rename(columns={"index":'word', 0: "count"}, inplace=True)
vocabluary_count.sort_values("count", ascending=False).head(500)

,word,count
44,женский,23676
14,смотреть,19143
159,мл,18187
96,чехол,15353
142,набор,13995
...,...,...
1656,окно,425
904,окрашивание,423
1389,портативный,423
1436,слип,423


In [46]:
#добавление в список стоп-слов
new_words=['женский', 'смотреть', 'мл', 'х', 'шт', 'мужской', 'цвет', 'чёрный', 'подарочный', 'девочка', 'штука', 'мм', 'принтом', 'подарок',
 'белый', 'размер', 'мальчик', 'с', 'м', 'л', '№', 'г', 'часы', 'вид', 'натуральный',
 'унисекс', 'год', 'рождение', 'универсальный', 'пара', 'цветной', 'сердце', 'рисунок', 'хлопковый', 'дизайн', 'красный', 'а', 'короткий', 'для',
 'ванна', 'длинный','надпись', 'круглый', 'упаковочный', 'искусственный', 'высокий', 'гр', 'душа', 'новый', 
 'устройство', 'век', 'хх', 'февраль', 'работа', 'новорождённый', 'зелёный', 'синий', 'вода', 'умывание', 'нижний', 'наращивание', 'дисплей', 'двойной',
 'зимний', 'разный', 'хсм', 'метр', 'тон', 'микс', 'фольгировать', 'в', 'больший', 'мягкий', 'белые', 'ты', 'я', 
 'одноразовый', 'тип', 'узор', 'ребёнок']
stopwords_ru_new = stopwords.words("russian")
for i in new_words:
    stopwords_ru_new.append(i)

In [121]:
#функция лемматизации с обновленным словарём стоп-слов
def lematiz2(text):
  new_text = []
  for word in text:
    if word not in stopwords_ru_new:
      word = morph.normal_forms(word)[0]      
      new_text.append(word)
  return new_text



Обработка текста

In [122]:
# обработка слов на полных данных
train['new_title2'] = train['title'].apply(regular).apply(tokeniz).apply(lematiz2).apply(string)

In [123]:
#обработка слов на "сбалансированных" данных
train_balance['new_title2'] = train_balance['title'].apply(regular).apply(tokeniz).apply(lematiz2).apply(string)

In [124]:
#обработка слов на данных без дубликатов
train_drop['new_title2'] = train_drop['title'].apply(regular).apply(tokeniz).apply(lematiz2).apply(string)
train_drop = train_drop.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Выбор модели



In [125]:
#Разбиение на обучающую и тестовую выборку полных данных 
x = train['new_title2']
y = train['category_id']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [126]:
#Разбиение на обучающую и тестовую выборку на "сбалансированных" данных
x_balance = train_balance['new_title2']
y_balance = train_balance['category_id']
X_train_balance, X_test_balance, y_train_balance, y_test_balance = train_test_split(x_balance, y_balance, test_size=0.33)

In [127]:
#Разбиение на обучающую и тестовую выборку на данных без дубликатов
x_drop = train_drop['new_title2']
y_drop = train_drop['category_id']
X_train_drop, X_test_drop, y_train_drop, y_test_drop = train_test_split(x_drop, y_drop, test_size=0.33)

In [128]:
#создание и обучение модели наивный байесовский классификатор
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
#Обучение и предсказание на полных данных
nb_all = nb.fit(X_train, y_train)
y_pred_all = nb_all.predict(X_test)

#Обучение и предсказание на "сбалансированных" данных
nb_balance = nb.fit(X_train_balance, y_train_balance)
y_pred_balance = nb_balance.predict(X_test_balance)

#Обучение и предсказание на данных без дупликатов
nb_drop = nb.fit(X_train_drop, y_train_drop)
y_pred_drop = nb_drop.predict(X_test_drop)
#оценка модели
print("Полные данные, модель НБ, f1_score:", f1_score(y_test, y_pred_all, average='micro'))
print("сбалансированные данные, модель НБ, f1_score:", f1_score(y_test_balance, y_pred_balance, average='micro'))
print("данные без дубликатов, модель НБ, f1_score:", f1_score(y_test_drop, y_pred_drop, average='micro'))

Полные данные, модель НБ, f1_score: 0.5901004917682275
сбалансированные данные, модель НБ, f1_score: 0.6346621455819038
данные без дубликатов, модель НБ, f1_score: 0.5620365052810308


In [130]:
#создание и обучение модели К ближйших соседей
kn = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', KNeighborsClassifier()),
              ])
#Обучение и предсказание на полных данных
kn_all = kn.fit(X_train, y_train)
y_pred_all = kn_all.predict(X_test)

#Обучение и предсказание на "сбалансированных" данных
kn_balance = kn.fit(X_train_balance, y_train_balance)
y_pred_balance = kn_balance.predict(X_test_balance)

#Обучение и предсказание на данных без дупликатов
kn_drop = kn.fit(X_train_drop, y_train_drop)
y_pred_drop = kn_drop.predict(X_test_drop)

#оценка модели
print("Полные данные, модель K_neigh, f1_score:", f1_score(y_test, y_pred_all, average='micro'))
print("сбалансированные данные, модель K_neigh, f1_score:", f1_score(y_test_balance, y_pred_balance, average='micro'))
print("данные без дубликатов, модель K_neigh, f1_score:", f1_score(y_test_drop, y_pred_drop, average='micro'))

Полные данные, модель K_neigh, f1_score: 0.7314838571734017
сбалансированные данные, модель K_neigh, f1_score: 0.6403506815089121
данные без дубликатов, модель K_neigh, f1_score: 0.7098429423955653


In [131]:
#создание и обучение линейной модели классификации
svm = Pipeline([('vect', CountVectorizer()),
               ('clf', SGDClassifier()),])

#Обучение и предсказание на полных данных
svm_all = svm.fit(X_train, y_train)
y_pred_all = svm_all.predict(X_test)

#Обучение и предсказание на "сбалансированных" данных
svm_balance = svm.fit(X_train_balance, y_train_balance)
y_pred_balance = svm_balance.predict(X_test_balance)

#Обучение и предсказание на данных без дупликатов
svm_drop = svm.fit(X_train_drop, y_train_drop)
y_pred_drop = svm_drop.predict(X_test_drop)
#оценка модели
print("Полные данные, модель svm, f1_score:", f1_score(y_test, y_pred_all, average='micro'))
print("сбалансированные данные, модель svm, f1_score:", f1_score(y_test_balance, y_pred_balance, average='micro'))
print("данные без дубликатов, модель svm, f1_score:", f1_score(y_test_drop, y_pred_drop, average='micro'))

Полные данные, модель svm, f1_score: 0.7703121659183237
сбалансированные данные, модель svm, f1_score: 0.7047984473643117
данные без дубликатов, модель svm, f1_score: 0.7624909485879797


Предсказание и создание parquet

In [132]:
#предобработка тестового файла (столбца title)
test['new_title'] = test['title'].apply(regular).apply(tokeniz).apply(lematiz2).apply(string)

In [133]:
y_pred_final = svm_all.predict(test['new_title'])

In [134]:
test['predicted_category_id'] = y_pred_final

In [135]:
final = test[['id', 'predicted_category_id']]

In [137]:
final.to_parquet('result.parquet')